### Objective: this notebook serves to fit HC behavioral data to model 2 (2 LR)

In [1]:
import numpy as np  

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import math
import os

from scipy.stats import norm
from scipy.optimize import minimize

import import_ipynb

In [2]:
os.getcwd()

'/Users/jackiebeltran/Documents/GitHub/RP_Learning/1_scripts/model_2'

#### load recoded data 

In [19]:
flist = glob.glob('../../0_data/healthy/*.csv')
print('HC participants in this dataset are: ' + str(len(flist)))

HC participants in this dataset are: 38


In [24]:
mlist = glob.glob('../../0_data/mdd/*.csv')
print('MDD participants in this dataset are: ' + str(len(flist)))

MDD participants in this dataset are: 38


###### All subjects

In [10]:
flist = glob.glob('../../0_data/N_76/*.csv')
print('participants in this dataset are: ' + str(len(flist)))

participants in this dataset are: 76


#### define functions

In [6]:
def m2_loglikelihood(training_params, actions, rewards, condition):

    r_alpha = training_params[0]
    p_alpha = training_params[1]
    beta = training_params[2]

    q_value = np.ones((3,2))*0.0
    T = len(actions)
    L = 0  
    
    for t in range(T):
        
        # compute choice probabilities of picking an action based on soft max 
        p = np.exp(beta * q_value[condition[t]-1,:]) / (np.exp(beta * q_value[condition[t]-1,:])).sum()

        # compute choice probability for actual choice based on the probability computed above by condition 
        choiceProb = p[actions[t]-1]

        # sum of the natural log of each individual choice probability to get the prob of a whole dataset (90 trials)
        L += np.log(choiceProb) 
    
        # update values with q learning, index for t and update by condition using the two separate learning rates
        if condition[t] == 1:
            q_value[condition[t]-1, actions[t]-1] = q_value[condition[t]-1, actions[t]-1] + r_alpha * (rewards[t] - q_value[condition[t]-1, actions[t]-1])
        elif condition[t] == 2:
            q_value[condition[t]-1, actions[t]-1] = q_value[condition[t]-1, actions[t]-1] + p_alpha * (rewards[t] - q_value[condition[t]-1, actions[t]-1])
    
    return -L   


In [7]:
def fit_RP_Learning(actions, rewards, condition):

    init_cond = np.array([np.random.uniform(0,1), # reward alpha
                          np.random.uniform(0,1), # punishment alpha
                         np.random.randint(1,50)]) # beta 
    
    bnds = [(1e-6,1), (1e-6,1), (1,50)]
    
    optimum_output = minimize(m2_loglikelihood, init_cond, args=(actions, rewards, condition), method='L-BFGS-B', bounds=bnds)
     
    return optimum_output, init_cond

In [8]:
def model2_BIC(actions, rewards, condition):
    
    init_cond = np.array([np.random.uniform(0,1), # reward alpha
                          np.random.uniform(0,1), # punishment alpha
                         np.random.randint(1,50)]) # beta 
     
    bnds = [(1e-6,1), (1e-6,1), (1,50)]
    km = len(bnds) # number of parameters fit in the model
    
    optimum_output = minimize(m2_loglikelihood, init_cond, args=(actions, rewards, condition), method='L-BFGS-B', bounds=bnds)
    
    neg_loglikelihood = optimum_output.fun
    
    BIC = km * np.log(len(actions)) + 2*neg_loglikelihood 
    
    return BIC

In [9]:
def fitting(filename):    
    
    parameters = [] # empty list to store results

    a = os.path.basename(filename)
    subID = os.path.splitext(a)[0]

    # load data & subset for columns of interest
    df = pd.read_csv(filename)

    col_name = ['sub_ID', 'actions','rewards', 'condition']   
    sub_df = df[col_name]

    # Obtain parameters (actions, rewards, conditions)  
    actions = sub_df['actions']
    reward = sub_df['rewards']
    condition = sub_df["condition"]

    # compute best fit parameters which comes from fitting function
    res, init_cond = fit_RP_Learning(actions, reward, condition) # optimizer

    initial_values = init_cond
    
    # save fitted parameters as an array 
    fit_params = np.array([res.x[0], res.x[1], res.x[2]])
    
    max_LL = m2_loglikelihood(fit_params, actions, reward, condition) 

    LL_per_trial = np.exp(-max_LL/len(actions))

    ## Model recovery             
    BIC_model2 = model2_BIC(actions, reward, condition) 

    results_df = pd.DataFrame({
    'subject_ID': subID,  
    'initial_reward_conditions': initial_values[0],
    'initial_punishment_conditions': initial_values[1],
    'initial_beta_conditions' : initial_values[2],
    'fit_reward_alpha': res.x[0],
    'fit_punishment_alpha': res.x[1],
    'fit_beta' : res.x[2],
    'log_likelihood': -max_LL,
    'BIC_model2': BIC_model2,
    'likelihood_per_trial': LL_per_trial}, 
    index=pd.Index(sub_df['sub_ID'].unique(), name='participant_id'))
        
    parameters.append(results_df)


    final_result_df = pd.concat(parameters, ignore_index=True)
    
    return final_result_df

#### fit the data

In [25]:
n_iter = 5
final_df = pd.DataFrame()

for j in np.arange(n_iter) + 1:
    
    for i in mlist: # FOR MDD
    #  for i in flist: # FOR ALL SUBJECTS
        iteration_results = fitting(i)
        
        # add iteration column 
        iteration_results['Iteration'] = j
                
        final_df = final_df.append(iteration_results, ignore_index=True)
        final_df = final_df.sort_values(by=["subject_ID", "Iteration"])

final_df

/var/folders/jc/p5x3b5x57nj1x3fy75x85xwr0000gn/T/ipykernel_16952/2124389525.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(iteration_results, ignore_index=True)
/var/folders/jc/p5x3b5x57nj1x3fy75x85xwr0000gn/T/ipykernel_16952/2124389525.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(iteration_results, ignore_index=True)
/var/folders/jc/p5x3b5x57nj1x3fy75x85xwr0000gn/T/ipykernel_16952/2124389525.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(iteration_results, ignore_index=True)
/var/folders/jc/p5x3b5x57nj1x3fy75x85xwr0000gn/T/ipykernel_16952/2124389525.py:13: FutureWarning: The frame.append method is deprecated and will be rem

,subject_ID,initial_reward_conditions,initial_punishment_conditions,initial_beta_conditions,fit_reward_alpha,fit_punishment_alpha,fit_beta,log_likelihood,BIC_model2,likelihood_per_trial,Iteration
0,401,0.761929,0.992035,33.0,0.400564,0.132609,8.455133,-37.290661,88.080751,0.660776,1
1,401,0.261853,0.387359,4.0,0.400567,0.132605,8.455384,-37.290661,88.080751,0.660776,2
2,401,0.406398,0.443551,23.0,0.400564,0.132604,8.455447,-37.290661,88.080751,0.660776,3
3,401,0.143224,0.857785,17.0,0.400565,0.132604,8.455401,-37.290661,88.080751,0.660776,4
4,401,0.215471,0.015683,27.0,0.400556,0.132605,8.455285,-37.290661,89.341134,0.660776,5
...,...,...,...,...,...,...,...,...,...,...,...
184,455,0.106702,0.832925,5.0,0.693353,0.064746,7.692783,-43.016337,99.532102,0.620047,1
185,455,0.091267,0.092768,25.0,0.008426,0.012253,50.000000,-44.459437,99.532102,0.610185,2
186,455,0.026543,0.360004,30.0,0.008426,0.012253,50.000000,-44.459437,99.532102,0.610185,3
187,455,0.081345,0.015270,15.0,0.032667,0.035516,14.993451,-44.586647,102.418303,0.609323,4


In [21]:
## FOR HEALTHY CONTROLS

final_df.to_csv('../../3_results/model_2/HC_fitting_results.csv') 

In [26]:
## FOR MDD 

final_df.to_csv('../../3_results/model_2/MDD_fitting_results.csv') 

In [12]:
### All subjects 
final_df.to_csv('../../3_results/model_2/N76_fitting_results.csv') 

#### keep greatest likelihood

In [27]:
df_sorted = final_df.sort_values(['subject_ID', 'likelihood_per_trial'], ascending=[True, False])
df_highest_likelihood = df_sorted.groupby('subject_ID').first().reset_index()
df_highest_likelihood

,subject_ID,initial_reward_conditions,initial_punishment_conditions,initial_beta_conditions,fit_reward_alpha,fit_punishment_alpha,fit_beta,log_likelihood,BIC_model2,likelihood_per_trial,Iteration
0,401,0.261853,0.387359,4.0,0.400567,0.132605,8.455384,-37.290661,88.080751,0.660776,2
1,402,0.296034,0.735612,12.0,0.477326,0.173945,3.667169,-45.570645,104.640718,0.602697,1
2,408,0.888023,0.763353,6.0,0.000001,0.000001,1.000000,-62.383247,138.265956,0.500000,4
3,409,0.968474,0.444753,10.0,0.011287,0.004338,50.000000,-43.137842,99.775113,0.619211,5
4,410,0.587989,0.226288,36.0,0.002828,0.019379,50.000000,-51.243911,115.143905,0.565878,3
5,413,0.941062,0.416180,1.0,0.372604,0.078903,30.445991,-30.236947,73.973323,0.714647,5
6,415,0.207327,0.734232,33.0,0.904114,0.021822,50.000000,-35.157606,83.814641,0.676624,2
7,416,0.934953,0.148102,20.0,0.043978,0.537191,5.964876,-42.036563,97.572554,0.626834,3
8,417,0.147810,0.781793,42.0,0.046942,0.848762,2.204035,-56.309741,126.118910,0.534906,5
9,418,0.699189,0.107451,15.0,0.376755,0.030578,35.726461,-34.947147,83.393724,0.678208,5


In [15]:
### All subjects 
df_highest_likelihood.to_csv('../../3_results/model_2/N76_parameters.csv') 

In [23]:
### HC
df_highest_likelihood.to_csv('../../3_results/model_2/HC_parameters.csv') 


In [28]:
### MDD

df_highest_likelihood.to_csv('../../3_results/model_2/MDD_parameters.csv')

### Extract RPEs 

In [57]:
os.getcwd()

'/Users/jackiebeltran/Documents/GitHub/RP_Learning/1_scripts/model_2'

In [56]:
def likelihood_mle_outputs(r_alpha, p_alpha, actions, rewards, condition):
        
    q_value = np.ones((3,2))*0.0
    T = len(actions)
    L = 0  
    RPE = []
    
    for t in range(T): # for every trial 
        
        # compute choice probabilities of picking an action based on soft max 
        # p = np.exp(beta * q_value[condition[t]-1,:]) / (np.exp(beta * q_value[condition[t]-1,:])).sum()

        # # compute choice probability for actual choice based on the probability computed above by condition 
        # choiceProb = p[actions[t]-1]

        # # sum of the natural log of each individual choice probability to get the prob of a whole dataset (90 trials)
        # L += np.log(choiceProb) 

        # update values with q learning, index for t and update by condition 
        if condition[t] == 1:
            rpe_r = rewards[t] - q_value[condition[t]-1, actions[t]-1]     
            q_value[condition[t]-1, actions[t]-1] = q_value[condition[t]-1, actions[t]-1] + r_alpha * (rpe_r)
            RPE.append(rpe_r)
            
        elif condition[t] == 2:
            rpe_p = rewards[t] - q_value[condition[t]-1, actions[t]-1]
            q_value[condition[t]-1, actions[t]-1] = q_value[condition[t]-1, actions[t]-1] + p_alpha * (rpe_p)
            RPE.append(rpe_p)
            
        elif condition[t] == 3: 
            rpe_n = 0 
            RPE.append(rpe_n)
        
    return RPE   


In [18]:
def lat_vars(filename):
    
    a = os.path.basename(filename)
    subID = os.path.splitext(a)[0]

    # load data & subset for columns of interest
    sub_df = pd.read_csv(filename)

    # Obtain vars of interest (actions, rewards, conditions)  
    condition = sub_df["condition"]
    actions = sub_df['actions']
    rewards = sub_df['rewards']
    
    r_alpha = sub_df['fit_reward_alpha'].iloc[0]
    
    p_alpha = sub_df['fit_punishment_alpha'].iloc[0]
    
    #fit_beta = sub_df['fit_beta'].iloc[0]
       
    # extract RPEs
    rpes = likelihood_mle_outputs(r_alpha, p_alpha, actions, rewards, condition)
    
    results_df = pd.DataFrame(rpes, columns = ['RPE'])
    results_df['sub_ID'] = subID
    results_df = results_df[['sub_ID', 'RPE']]
    
    results_df.to_csv(f'../../3_results/model_2/RPEs/{subID}_RPE.csv')

##### load data

In [58]:
### Healthy Controls 
df1_files = glob.glob('../../0_data/healthy/*.csv') 
df_highest_likelihood = pd.read_csv(f'/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/HC_parameters.csv')

In [44]:
### MDD 
df1_files = glob.glob('../../0_data/mdd/*.csv') 
df_highest_likelihood = pd.read_csv(f'/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/MDD_parameters.csv')

In [59]:
df_highest_likelihood.head(5)

,Unnamed: 0,subject_ID,initial_reward_conditions,initial_punishment_conditions,initial_beta_conditions,fit_reward_alpha,fit_punishment_alpha,fit_beta,log_likelihood,BIC_model2,likelihood_per_trial,Iteration
0,0,501,0.613797,0.668543,4.0,0.646656,0.119117,6.709861,-41.818905,98.214834,0.628352,5
1,1,502,0.208690,0.986033,15.0,1.000000,0.783051,3.021224,-48.528095,114.427152,0.583214,5
2,2,503,0.521664,0.093482,11.0,0.340769,0.015024,50.000000,-37.243030,87.985488,0.661126,5
3,3,504,0.192704,0.565117,19.0,0.109320,1.000000,5.650866,-37.352489,88.204406,0.660322,4
4,4,506,0.513802,0.488712,47.0,0.121509,0.323182,6.670178,-34.830305,83.160039,0.679089,3


In [60]:
# empty list to create a single dataframe with all subjects behavioral data
df1_dataframes = []

# load and append each df1 dataframe to the list
for file in df1_files:
    
    df = pd.read_csv(file)
    
    col_name = ['sub_ID', 'actions','rewards', 'condition']   
    df = df[col_name]    
    
    df1_dataframes.append(df)

combined_df = pd.concat(df1_dataframes, ignore_index=True)

In [62]:
combined_df.head(10)

,sub_ID,actions,rewards,condition
0,545,1,1,1
1,545,2,0,3
2,545,1,0,2
3,545,1,0,3
4,545,2,-1,2
5,545,1,1,1
6,545,1,1,1
7,545,1,1,1
8,545,1,1,1
9,545,1,0,2


In [63]:
# load parameters with greatest likelihood

cols = ['sub_ID', 'fit_reward_alpha', 'fit_punishment_alpha', 'fit_beta']
df_highest_likelihood.rename(columns = {'subject_ID':'sub_ID'}, inplace = True)

df_highest_likelihood = df_highest_likelihood[cols]

In [64]:
df_highest_likelihood.head(5)

# subset df_highest_likelihood dataframe for a specific sub_ID
df_highest_likelihood.loc[df_highest_likelihood['sub_ID'] == 545]



,sub_ID,fit_reward_alpha,fit_punishment_alpha,fit_beta
33,545,0.318487,0.444252,2.80489


In [65]:
# create one dataframe with behavioral data & parameters for each subject

params_df = pd.merge(combined_df, df_highest_likelihood)
params_df

,sub_ID,actions,rewards,condition,fit_reward_alpha,fit_punishment_alpha,fit_beta
0,545,1,1,1,0.318487,0.444252,2.804890
1,545,2,0,3,0.318487,0.444252,2.804890
2,545,1,0,2,0.318487,0.444252,2.804890
3,545,1,0,3,0.318487,0.444252,2.804890
4,545,2,-1,2,0.318487,0.444252,2.804890
...,...,...,...,...,...,...,...
3415,548,2,0,3,0.000556,0.032636,11.434186
3416,548,1,1,1,0.000556,0.032636,11.434186
3417,548,1,0,2,0.000556,0.032636,11.434186
3418,548,1,1,1,0.000556,0.032636,11.434186


In [66]:
params_df.loc[params_df['sub_ID'] == 545]

,sub_ID,actions,rewards,condition,fit_reward_alpha,fit_punishment_alpha,fit_beta
0,545,1,1,1,0.318487,0.444252,2.80489
1,545,2,0,3,0.318487,0.444252,2.80489
2,545,1,0,2,0.318487,0.444252,2.80489
3,545,1,0,3,0.318487,0.444252,2.80489
4,545,2,-1,2,0.318487,0.444252,2.80489
...,...,...,...,...,...,...,...
85,545,1,-1,2,0.318487,0.444252,2.80489
86,545,1,0,3,0.318487,0.444252,2.80489
87,545,1,1,1,0.318487,0.444252,2.80489
88,545,1,0,3,0.318487,0.444252,2.80489


In [51]:
# save dataframe with participant data & parameters into individual subject dataframes

grouped = params_df.groupby('sub_ID')

output_directory = '/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params'

for group_name, group_df in grouped:
    file_name = os.path.join(output_directory, f'{group_name}.csv')  # Generate a unique file name for each group
    group_df.to_csv(file_name, index=False)

##### extract RPEs

In [67]:
hlist = glob.glob('/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/5*.csv')
mlist = glob.glob('/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/4*.csv')

print(len(hlist))
print(len(mlist))

38
38


In [55]:
for name in mlist:
# for name in hlist:
    print(name)
    lat_vars(name)

/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/431.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/425.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/419.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/418.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/424.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/430.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/432.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/427.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/423.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/436.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/final_params/422.csv
/Users/jackiebeltran/